<a href="https://colab.research.google.com/github/flozgom/Machine_Learning_Healthcare/blob/main/MITx_Chart_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Chart Review</h1>



In [ ]:
  #@title Importing libraries
from google.colab import auth
from google.colab import widgets

from collections import Counter
import numpy as np
import pandas as pd
import pickle

In [ ]:
#@title Authenticating User 
auth.authenticate_user()
!gsutil cp gs://mlhc-mimic/chartreview.pkl ./

In [ ]:
##Importing data
with open('chartreview.pkl', 'rb') as f:
  notes       = pickle.load(f)
  backgrounds = pickle.load(f)

In [ ]:
#@title Defining helper functions (view_background(), visualize_notes()) for visualizing data
def view_background(hadm_id):
  display(backgrounds[backgrounds.index==hadm_id].T)


def visualize_notes(hadm_id):
  # When did this patient arrive (useful for getting first 48 hours)
  admittime = notes[notes.hadm_id==hadm_id].admittime.values[0]

  # Get the notes for this patient
  notes_subject = notes.loc[notes.hadm_id==hadm_id]

  # How many notes for each category?
  category_counts = Counter(notes_subject.category.values)
  category_sorted = sorted(category_counts.keys(), 
                            key=lambda t:category_counts[t], reverse=True)

  # Outer tab is for different category of notes
  outer_tab = widgets.TabBar(category_sorted, location='top')
  for category in category_sorted:
    with outer_tab.output_to(category):
      notes_cat = notes_subject.loc[notes_subject.category==category]
      titles = []
      for num,(i,row) in enumerate(notes_cat.iterrows()):
        # Format the text with additional metadata
        time_offset = (row.charttime - admittime).total_seconds()/3600.
        time_offset = int(time_offset) if not np.isnan(time_offset) else 'n/a'
          
        # Only first 48 hours of data
        titles += ['%s Note #%d (%s Hours)' % (category,num,time_offset)]

      # Inner tab is for each note in a category
      inner_tab = widgets.TabBar(titles, location='start')
      for i in range(len(titles)):
        with inner_tab.output_to(titles[i]):
          print(notes_cat.iloc[i]["text"])
      

<h1>Patient 1</h1>



In [ ]:
##Overview of patient with admission_id = 142861
hadm_id = 194001
view_background(hadm_id)

In [ ]:
##Visualizing Clinical Notes for 142861
tab = visualize_notes(hadm_id)

**<h1>Patient 2</h1>**


In [ ]:
##Overview of patient with admission_id = 194001
hadm_id_2 = 194001
view_background(hadm_id_2)

In [ ]:
##Visualizing Clinical Notes for 194001
visualize_notes(194001)